# Visualising Eye Movement

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import numpy as np

import matplotlib
matplotlib.use("Agg") 

In [2]:
import sys
import os

sys.path.append(os.path.abspath("../src"))  # Add folder_a to sys.path
from config import *

In [3]:
experiment = "EVIL_BASTARD"
participant_ids = pd.read_parquet(
        f"{CLEANED_DIR}/{experiment}_samples.pq", 
        columns=["participant_id"]
    )

participant_ids = participant_ids["participant_id"].unique()

In [6]:
# Read data for participant
participant_id = participant_ids[3]
filters = [
    ('participant_id', '=', participant_id),
]

samples_df = pd.read_parquet(
    f"{CLEANED_DIR}/{experiment}_samples.pq",
    filters=filters
    )

events_df = pd.read_parquet(
    f"{CLEANED_DIR}/{experiment}_events.pq",
    filters=filters
    )

In [12]:
events_df.head()

,experiment,participant_id,trial_id,time,event,eye,start_time,end_time,duration,x,...,end_y,amplitude,peak_velocity,angle,speed,target_x,target_y,colour,stimulus_x,stimulus_y
0,EVIL_BASTARD,121,0.0,3096638.0,TRIALID,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,NaN,NaN
1,EVIL_BASTARD,121,0.0,3096644.0,START,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,NaN,NaN
2,EVIL_BASTARD,121,0.0,3096651.0,SFIX,L,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,NaN,NaN
3,EVIL_BASTARD,121,0.0,3096651.0,SFIX,R,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,NaN,NaN
4,EVIL_BASTARD,121,0.0,3097001.0,SSACC,L,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,NaN,NaN


In [13]:
events_df["event"].unique()

array(['TRIALID', 'START', 'SFIX', 'SSACC', 'SBLINK', 'TRIAL_VAR_DATA',
       'END', 'EFIX', 'ESACC', 'EBLINK'], dtype=object)

In [7]:
# Extract fixpoints
fixpoints = events_df[events_df["event"]=="FIXPOINT"].loc[:,["trial_id", "time", "event", "stimulus_x", "stimulus_y"]]

In [10]:
trial_id=1

sample_df = samples_df.loc[samples_df["trial_id"]==trial_id,:]

# Extract fixpoints
event_df = events_df.loc[events_df["trial_id"]==trial_id,:]
fixpoints_df = event_df[event_df["event"]=="FIXPOINT"].loc[:,["trial_id", "time", "event", "colour", "stimulus_x", "stimulus_y"]]

# Insert fixpoints in sample data
# Ensure you are modifying actual copies
sample_df = sample_df.copy()
fixpoints_df = fixpoints_df.copy()

# Force types
sample_df["time"] = sample_df["time"].astype("float64")
fixpoints_df["time"] = fixpoints_df["time"].astype("float64")

# Make sure both DataFrames are sorted by time
sample_df = sample_df.sort_values("time")
fixpoints_df = fixpoints_df.sort_values("time")

# Rename 'colour' column to 'fixpoint' so it's ready to merge
fixpoints_df = fixpoints_df.rename(columns={"colour": "fixpoint"})

# Perform a backward-looking join: for each row in sample_df, find the most recent fixpoint time
sample_df = pd.merge_asof(
    sample_df,
    fixpoints_df,
    on="time",
    direction="backward"
)

sample_df["fixpoint"] = sample_df["fixpoint"].map({RED:"red", GREEN:"green", BLUE:"blue", WHITE:"white"})

In [11]:
x = sample_df.copy()
x_sampled = x[x.index % 100 == 0]

# Apply Seaborn style
sns.set(style="whitegrid")

fig, ax = plt.subplots(1, 1, figsize = (6, 6))

# Initialize the scatter plot object
scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_fixpoint = ax.scatter([], [], label="Fixpoint", alpha=0.6, s=50)  # 's' is for size, 'c' is for color

x_min, x_max = 0, 1920 
y_min, y_max = 0, 1080 
ax.set_xlim([x_min, x_max]) 
ax.set_ylim([y_min, y_max]) 

# Add legend to distinguish between the three points
ax.legend()

# Initialize function: this is called to create the background of each frame
# def init():
#     # Set empty offsets initially
#     scatter_right.set_offsets([])
#     scatter_left.set_offsets([])  
#     scatter_fixpoint.set_offsets([])  
    
#     return scatter_left, scatter_right, scatter_fixpoint,

def animate(i):
    # Update the scatter plot data for each frame
    
    # Update coordinates
    coords_right = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_right"].iloc[i]]
    coords_left = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_left"].iloc[i]] 
    coords_fixpoint = np.c_[x_sampled["stimulus_x"].iloc[i], x_sampled["stimulus_y"].iloc[i]] 

    scatter_right.set_offsets(coords_right)    
    scatter_left.set_offsets(coords_left)
    scatter_fixpoint.set_offsets(coords_fixpoint)

    # Update colour based on fixpoint
    fixpoint_color = x_sampled["fixpoint"].iloc[i]
    if pd.isna(fixpoint_color):
        fixpoint_color = "gray"
    elif fixpoint_color == "white":
        fixpoint_color = "black"
    scatter_fixpoint.set_color([fixpoint_color])

    return scatter_right, scatter_left, scatter_fixpoint

anim = animation.FuncAnimation(
    fig, 
    animate, 
    frames = len(x_sampled), 
    interval=100,
    blit = False
)

anim.save(f"{experiment}_{participant_id}_{trial_id}.mp4", writer="ffmpeg", fps=30)



/tmp/ipykernel_1750/3347012537.py:10: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
/tmp/ipykernel_1750/3347012537.py:11: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
